In [1]:
"""
Load the dataset and select relevant columns.
"""

import pandas as pd
import os

# Gets the absolute path to the data folder.
dirname = os.path.abspath('')
filename = os.path.join(dirname, '..', '..', 'data', 'all_data.csv')

# Load the data
df = pd.read_csv(filename)

# Only keeps the cols that will be used
df = df[['comment_text', 'toxicity', 'severe_toxicity']]

# Something was wrong in the dataset and only certain intervals were labeled.
df = df.iloc[list(range(0, 10565+1)) + list(range(65650, 76241+1)) + list(range(131626, 142099+1)) + list(range(262634, 273124+1)) + list(range(328456, 338866+1)) + list(range(394286, 404766+1)) + list(range(460042, 470469+1)) + list(range(525922, 536570+1)) + list(range(591591, 602307+1))]


In [4]:
"""
Performing autocorrect on dataset.
"""
from autocorrect import Speller

# fast=False is preferable but is way to slow for these purposes, fast=True means that words with more than one error won't be fixed.
spell = Speller(fast=True)

# .map() performed sligthly better than .apply() in tests.
#df['comment_text'] = df['comment_text'].astype(str).apply(spell)
df['comment_text'] = df['comment_text'].astype(str).map(spell)


In [ ]:
"""
Remove punctuation, convert to lowercase.
"""

df['comment_text'] = df['comment_text'].str.replace(r'[^\w\s]+', '', regex=True)  # remove punctuation
df['comment_text'] = df['comment_text'].apply(lambda x : x.lower())  # make lowercase


In [ ]:
"""
Download neccessary nltk data.
"""
from nltk import download

download("wordnet")
download("omw-1.4")
download("stopwords")


In [3]:
"""
Tokenize messages
"""

import re

df['comment_text'] = df['comment_text'].apply(lambda x : re.split(r'\s+', x))


In [5]:
"""
Remove Stop words
"""

from nltk.corpus import stopwords

stop_words = set(map(lambda x : re.sub(r'[^\w\s]+', '', x), stopwords.words('english')))  # Loads nltk stopwords and removes punctuation

df['comment_text'] = df['comment_text'].apply(lambda x : [word for word in x if word not in stop_words])


In [6]:
"""
Lemmatize data
"""

from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

df['comment_text'] = df['comment_text'].apply(lambda x : [wnl.lemmatize(wnl.lemmatize(word), pos='v') for word in x])  # Lemmatizes each word both as a noun and a verb


In [7]:
"""
Export the dataframe to a csv file.
"""

import os

# Gets the absolute path to the file.
dirname = os.path.abspath('')
filename = os.path.join(dirname, '..', '..', 'data', 'all_data_proccessed.csv')

df.to_csv(path_or_buf=filename)
